In [1]:
import tkinter as tk
from tkinter import scrolledtext
import mysql.connector
from difflib import get_close_matches

In [2]:
# Predefined dictionary mapping user inputs to SQL queries
conversation = {
    "Hi":"SELECT response FROM chatbot_responses WHERE  input='Hi'",
    "hello":"SELECT response FROM chatbot_responses WHERE  input='hello'",
    "hey":"SELECT response FROM chatbot_responses WHERE  input='hey'",
    "Hey there!":"SELECT response FROM chatbot_responses WHERE  input='Hey there!'",
    "hey you":"SELECT response FROM chatbot_responses WHERE  input='hey you'",
    
    "Ok":"SELECT response FROM chatbot_responses WHERE  input='Ok'",
    "Thank you!":"SELECT response FROM chatbot_responses WHERE  input='Thank you!'",
    "Ok! Thank you":"SELECT response FROM chatbot_responses WHERE  input='Ok! Thank you'",
    
    "tell me about the dataset":"SELECT response FROM chatbot_responses WHERE  input='tell me about the dataset'",
    "tell me about the database":"SELECT response FROM chatbot_responses WHERE  input='tell me about the database'",
    "Could you please tell me about the dataset":"SELECT response FROM chatbot_responses WHERE  input='Could you please tell me about the dataset'",
    "could you please tell me about the database":"SELECT response FROM chatbot_responses WHERE  input='could you please tell me about the database'",
    "Could you provide information regarding the database":"SELECT response FROM chatbot_responses WHERE  input='Could you provide information regarding the database'",
    "Give me information about the database":"SELECT response FROM chatbot_responses WHERE  input='Give me information about the database'",
    "describe the database":"SELECT response FROM chatbot_responses WHERE  input='could you tell me about the dataset'",
    "describe the dataset":"SELECT response FROM chatbot_responses WHERE  input='could you tell me about the dataset'",
    "what is the dataset about?":"SELECT response FROM chatbot_responses WHERE  input='could you tell me about the dataset'",
    "what is the database about?":"SELECT response FROM chatbot_responses WHERE  input='could you tell me about the dataset'",
    
    "Can you show me the dataset?":"SELECT * FROM retail_sales;",
    "Can you give me the database?":"SELECT * FROM retail_sales;",
    "Show me the database":"SELECT * FROM retail_sales;",
    "provide me the database":"SELECT * FROM retail_sales;",
    "Provide me the dataset":"SELECT * FROM retail_sales;",
    
    "number of female customers":"SELECT COUNT(*) female_customers FROM retail_sales WHERE gender='Female';",
    "no. of female customers":"SELECT COUNT(*) female_customers FROM retail_sales WHERE gender='Female';",
    "count of female custoers":"SELECT COUNT(*) female_customers FROM retail_sales WHERE gender='Female';",
    
    "number of male customers":"SELECT COUNT(*) male_customers FROM retail_sales WHERE gender='Male';",
    "no. of male customers":"SELECT COUNT(*) male_customers FROM retail_sales WHERE gender='Male';",
    "count of male custoers":"SELECT COUNT(*) male_customers FROM retail_sales WHERE gender='Male';",   
    
    "what are all the product categories available?":"SELECT DISTINCT(product_category) product_category FROM retail_sales;",
    "what are all the product categories available":"SELECT DISTINCT(product_category) product_category FROM retail_sales;",
    "show me the product categories":"SELECT DISTINCT(product_category) product_category FROM retail_sales;",
    "give me the product categories":"SELECT DISTINCT(product_category) product_category FROM retail_sales;",
    "product categories":"SELECT DISTINCT(product_category) product_category FROM retail_sales;",
    "what are all the product categories available":"SELECT DISTINCT(product_category) product_category FROM retail_sales;",
    "what are all the product categories available?":"SELECT DISTINCT(product_category) product_category FROM retail_sales;",
    "what are the product categories":"SELECT DISTINCT(product_category) product_category FROM retail_sales;",
    "what are the product categories?":"SELECT DISTINCT(product_category) product_category FROM retail_sales;",
    
    "number of customers in the database":"SELECT COUNT(DISTINCT(customer_id)) count_of_customers FROM retail_sales;",
    "how many customer details are there":"SELECT COUNT(DISTINCT(customer_id)) count_of_customers FROM retail_sales;",
    "how many customer details are there in the database":"SELECT COUNT(DISTINCT(customer_id)) count_of_customers FROM retail_sales;",
    "number of records in the database":"SELECT COUNT(DISTINCT(customer_id)) count_of_customers FROM retail_sales;",
    "no. of customers in the database":"SELECT COUNT(DISTINCT(customer_id)) count_of_customers FROM retail_sales;",
    "no. of records in the database":"SELECT COUNT(DISTINCT(customer_id)) count_of_customers FROM retail_sales;",
    "how many records are there":"SELECT COUNT(DISTINCT(customer_id)) count_of_customers FROM retail_sales;",
    "how many records are there in the database":"SELECT COUNT(DISTINCT(customer_id)) count_of_customers FROM retail_sales;",
    "how many records are there?":"SELECT COUNT(DISTINCT(customer_id)) count_of_customers FROM retail_sales;",
    "how many records are there in the database?":"SELECT COUNT(DISTINCT(customer_id)) count_of_customers FROM retail_sales;",
    "how many customer details are there?":"SELECT COUNT(DISTINCT(customer_id)) count_of_customers FROM retail_sales;",
    "how many customer details are there in the database?":"SELECT COUNT(DISTINCT(customer_id)) count_of_customers FROM retail_sales;",
    "number of records":"SELECT COUNT(DISTINCT(customer_id)) count_of_customers FROM retail_sales;",
    "no. of records":"SELECT COUNT(DISTINCT(customer_id)) count_of_customers FROM retail_sales;",
    "number of customers":"SELECT COUNT(DISTINCT(customer_id)) count_of_customers FROM retail_sales;",
    "no. of customers":"SELECT COUNT(DISTINCT(customer_id)) count_of_customers FROM retail_sales;",
    
    "what is the total amount earned":"SELECT SUM(total_amount) total_sale_price FROM retail_sales;",
    "what is the total amount earned?":"SELECT SUM(total_amount) total_sale_price FROM retail_sales;",
    "what is the total amount":"SELECT SUM(total_amount) total_sale_price FROM retail_sales;",
    "what is the total amount?":"SELECT SUM(total_amount) total_sale_price FROM retail_sales;",
    "give me the total amount ":"SELECT SUM(total_amount) total_sale_price FROM retail_sales;",
    "give me the total amount earned from sales":"SELECT SUM(total_amount) total_sale_price FROM retail_sales;",
    "what is the total amount earned from sales":"SELECT SUM(total_amount) total_sale_price FROM retail_sales;",
    "what is the total amount earned from sales?":"SELECT SUM(total_amount) total_sale_price FROM retail_sales;",
    "total amount earned":"SELECT SUM(total_amount) total_sale_price FROM retail_sales;",
    "total amount earned from sales":"SELECT SUM(total_amount) total_sale_price FROM retail_sales;",
    
    "what is the maximum amount earned per day":"SELECT MAX(total_amount) max_amount FROM retail_sales;",
    "what is the maximum amount earned per day?":"SELECT MAX(total_amount) max_amount FROM retail_sales;",
    "maximum amount earned per day":"SELECT MAX(total_amount) max_amount FROM retail_sales;",
    "maximum amount earned from one day sale":"SELECT MAX(total_amount) max_amount FROM retail_sales;",
    
    "what is the minimum amount earned per day":"SELECT MIN(total_amount) min_amount FROM retail_sales;",
    "what is the minimum amount earned per day?":"SELECT MIN(total_amount) min_amount FROM retail_sales;",
    "minimum amount earned per day":"SELECT MIN(total_amount) min_amount FROM retail_sales;",
    "minimum amount earned from one day sale":"SELECT MIN(total_amount) min_amount FROM retail_sales;",
    
    "no. of products sold in clothing category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Clothing' GROUP BY product_category;",
    "number of products sold in clothing category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Clothing' GROUP BY product_category;",
    "total products sold in clothing category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Clothing' GROUP BY product_category;",
    "total no. of products sold in clothing category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Clothing' GROUP BY product_category;",
    "total number of products sold in clothing category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Clothing' GROUP BY product_category;",
    "how many products have been sold in clothing category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Clothing' GROUP BY product_category;",
    "how many products have been sold in clothing category?":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Clothing' GROUP BY product_category;",
    "how many products have been sold in clothing category on total":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Clothing' GROUP BY product_category;",
    "total sold products count in clothing category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Clothing' GROUP BY product_category;",

    "no. of products sold in electronics category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Electronics' GROUP BY product_category;",
    "number of products sold in electronics category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Electronics' GROUP BY product_category;",
    "total products sold in electronics category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Electronics' GROUP BY product_category;",
    "total no. of products sold in electronics category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Electronics' GROUP BY product_category;",
    "total number of products sold in electronics category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Electronics' GROUP BY product_category;",
    "how many products have been sold in electronics category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Electronics' GROUP BY product_category;",
    "how many products have been sold in electronics category?":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Electronics' GROUP BY product_category;",
    "how many products have been sold in electronics category on total":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Electronics' GROUP BY product_category;",
    "total sold products count in electronics category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Electronics' GROUP BY product_category;",
    
    "no. of products sold in beauty category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Beauty' GROUP BY product_category;",
    "number of products sold in beauty category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Beauty' GROUP BY product_category;",
    "total products sold in beauty category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Beauty' GROUP BY product_category;",
    "total no. of products sold in beauty category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Beauty' GROUP BY product_category;",
    "total number of products sold in beauty category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Beauty' GROUP BY product_category;",
    "how many products have been sold in beauty category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Beauty' GROUP BY product_category;",
    "how many products have been sold in beauty category?":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Beauty' GROUP BY product_category;",
    "how many products have been sold in beauty category on total":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Beauty' GROUP BY product_category;",
    "total sold products count in beauty category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales WHERE product_category='Beauty' GROUP BY product_category;",
    
    "category-wise show me the no. of products sold":"SELECT product_category, SUM(quantity) quantity FROM retail_sales GROUP BY product_category ORDER BY quantity DESC;",
    "category-wise show me the number of products sold":"SELECT product_category, SUM(quantity) quantity FROM retail_sales GROUP BY product_category ORDER BY quantity DESC;",
    "show me the number of products sold in each category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales GROUP BY product_category ORDER BY quantity DESC;",
    "show me the number of products sold category-wise":"SELECT product_category, SUM(quantity) quantity FROM retail_sales GROUP BY product_category ORDER BY quantity DESC;",
    "show me the no. of products sold in each category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales GROUP BY product_category ORDER BY quantity DESC;",
    "show me the no. of products sold category-wise":"SELECT product_category, SUM(quantity) quantity FROM retail_sales GROUP BY product_category ORDER BY quantity DESC;",
    "no. of products sold in each category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales GROUP BY product_category ORDER BY quantity DESC;",
    "number of products sold in each category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales GROUP BY product_category ORDER BY quantity DESC;",
    "how many products have been sold in each category?":"SELECT product_category, SUM(quantity) quantity FROM retail_sales GROUP BY product_category ORDER BY quantity DESC;",
    "how many products have been sold in each category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales GROUP BY product_category ORDER BY quantity DESC;",
    "category-wise give me the no. of products sold":"SELECT product_category, SUM(quantity) quantity FROM retail_sales GROUP BY product_category ORDER BY quantity DESC;",
    "category-wise give me the number of products sold":"SELECT product_category, SUM(quantity) quantity FROM retail_sales GROUP BY product_category ORDER BY quantity DESC;",
    "give me the number of products sold in each category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales GROUP BY product_category ORDER BY quantity DESC;",
    "give me the number of products sold category-wise":"SELECT product_category, SUM(quantity) quantity FROM retail_sales GROUP BY product_category ORDER BY quantity DESC;",
    "give me the no. of products sold in each category":"SELECT product_category, SUM(quantity) quantity FROM retail_sales GROUP BY product_category ORDER BY quantity DESC;",
    "give me the no. of products sold category-wise":"SELECT product_category, SUM(quantity) quantity FROM retail_sales GROUP BY product_category ORDER BY quantity DESC;",
    
    "amount earned from the sale of clothings":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Clothing' GROUP BY product_category;",
    "what is the amount earned from the sale of clothings":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Clothing' GROUP BY product_category;",
    "what is the amount earned from the sale of clothings?":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Clothing' GROUP BY product_category;",
    "amount earned from clothings":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Clothing' GROUP BY product_category;",
    "what is the amount earned from clothings":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Clothing' GROUP BY product_category;",
    "what is the amount earned from clothings?":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Clothing' GROUP BY product_category;",
    "sale amount of clothings category":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Clothing' GROUP BY product_category;",
    "total sale amount of clothings category":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Clothing' GROUP BY product_category;",
    "what is the sale amount of clothings category":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Clothing' GROUP BY product_category;",
    "what is the total sale amount of clothings category":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Clothing' GROUP BY product_category;",
    "what is the sale amount of clothings category?":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Clothing' GROUP BY product_category;",
    "what is the total sale amount of clothings category?":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Clothing' GROUP BY product_category;",
    
    "amount earned from the sale of electronics":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Electronics' GROUP BY product_category;",
    "what is the amount earned from the sale of electronics":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Electronics' GROUP BY product_category;",
    "what is the amount earned from the sale of electronics?":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Electronics' GROUP BY product_category;",
    "amount earned from electronics":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Electronics' GROUP BY product_category;",
    "what is the amount earned from electronics":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Electronics' GROUP BY product_category;",
    "what is the amount earned from electronics?":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Electronics' GROUP BY product_category;",
    "sale amount of electronics category":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Electronics' GROUP BY product_category;",
    "total sale amount of electronics category":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Electronics' GROUP BY product_category;",
    "what is the sale amount of electronics category":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Electronics' GROUP BY product_category;",
    "what is the total sale amount of electronics category":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Electronics' GROUP BY product_category;",
    "what is the sale amount of electronics category?":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Electronics' GROUP BY product_category;",
    "what is the total sale amount of electronics category?":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Electronics' GROUP BY product_category;",
    
    "amount earned from the sale of beauty producsts":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Beauty' GROUP BY product_category;",
    "what is the amount earned from the sale of beauty products":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Beauty' GROUP BY product_category;",
    "what is the amount earned from the sale of beauty products?":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Beauty' GROUP BY product_category;",
    "amount earned from beauty products":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Beauty' GROUP BY product_category;",
    "what is the amount earned from beauty products":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Beauty' GROUP BY product_category;",
    "what is the amount earned from beauty products?":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Beauty' GROUP BY product_category;",
    "sale amount of beauty category":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Beauty' GROUP BY product_category;",
    "total sale amount of beauty category":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Beauty' GROUP BY product_category;",
    "what is the sale amount of beauty category":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Beauty' GROUP BY product_category;",
    "what is the total sale amount of beauty category":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Beauty' GROUP BY product_category;",
    "what is the sale amount of beauty category?":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Beauty' GROUP BY product_category;",
    "what is the total sale amount of beauty category?":"SELECT product_category, SUM(total_amount) amount FROM retail_sales WHERE product_category='Beauty' GROUP BY product_category;",
    
    "what is the total amount earned from each category?":"SELECT product_category, SUM(total_amount) amount FROM retail_sales GROUP BY product_category ORDER BY amount DESC;",
    "give me the total amount from each category":"SELECT product_category, SUM(total_amount) amount FROM retail_sales GROUP BY product_category ORDER BY amount DESC;",
    "what is the total amount earned from sales of each category?":"SELECT product_category, SUM(total_amount) amount FROM retail_sales GROUP BY product_category ORDER BY amount DESC;",
    
    "show me the days when the sale is highest":"SELECT date, total_amount FROM retail_sales WHERE total_amount = (SELECT MAX(total_amount) FROM retail_sales);",
    "when we have maximum sale with highest amount":"SELECT date, total_amount FROM retail_sales WHERE total_amount = (SELECT MAX(total_amount) FROM retail_sales);",
    "days when the earnings is the highest":"SELECT date, total_amount FROM retail_sales WHERE total_amount = (SELECT MAX(total_amount) FROM retail_sales);",
    "days when the earnings is the most":"SELECT date, total_amount FROM retail_sales WHERE total_amount = (SELECT MAX(total_amount) FROM retail_sales);",
    "days when the sale is in peak":"SELECT date, total_amount FROM retail_sales WHERE total_amount = (SELECT MAX(total_amount) FROM retail_sales);",
    "days of highest amount of sale":"SELECT date, total_amount FROM retail_sales WHERE total_amount = (SELECT MAX(total_amount) FROM retail_sales);",
    "days of highest sale":"SELECT date, total_amount FROM retail_sales WHERE total_amount = (SELECT MAX(total_amount) FROM retail_sales);",
    "days when the sale is highest":"SELECT date, total_amount FROM retail_sales WHERE total_amount = (SELECT MAX(total_amount) FROM retail_sales);",
    "days when the earnings is highest":"SELECT date, total_amount FROM retail_sales WHERE total_amount = (SELECT MAX(total_amount) FROM retail_sales);",
    "days of highest earning":"SELECT date, total_amount FROM retail_sales WHERE total_amount = (SELECT MAX(total_amount) FROM retail_sales);",
    
    "customers who had purchased for highest amount":"SELECT DISTINCT(customer_id), total_amount FROM retail_sales WHERE total_amount = (SELECT MAX(total_amount) FROM retail_sales);",
    "customers who purchased for highest amount":"SELECT DISTINCT(customer_id), total_amount FROM retail_sales WHERE total_amount = (SELECT MAX(total_amount) FROM retail_sales);",
    "show me the customers who had purchased for the highest amount":"SELECT DISTINCT(customer_id), total_amount FROM retail_sales WHERE total_amount = (SELECT MAX(total_amount) FROM retail_sales);",
    "customer id who purchased for highest amount":"SELECT DISTINCT(customer_id), total_amount FROM retail_sales WHERE total_amount = (SELECT MAX(total_amount) FROM retail_sales);",
    "customers purchased for highest amount":"SELECT DISTINCT(customer_id), total_amount FROM retail_sales WHERE total_amount = (SELECT MAX(total_amount) FROM retail_sales);",
    
    "amount earned from the purchase of male customers":"SELECT gender, SUM(total_amount) amount FROM retail_sales WHERE gender='Male' GROUP BY gender;",
    "what is the amount earned from the purchase of male customers":"SELECT gender, SUM(total_amount) amount FROM retail_sales WHERE gender='Male' GROUP BY gender;",
    "what is the amount earned from the purchase of male customers?":"SELECT gender, SUM(total_amount) amount FROM retail_sales WHERE gender='Male' GROUP BY gender;",
    "amount earned from the purchase of male customers":"SELECT gender, SUM(total_amount) amount FROM retail_sales WHERE gender='Male' GROUP BY gender;",
    "what is the amount earned from the purchase of male customers":"SELECT gender, SUM(total_amount) amount FROM retail_sales WHERE gender='Male' GROUP BY gender;",
    "what is the amount earned from the purchase of male customers?":"SELECT gender, SUM(total_amount) amount FROM retail_sales WHERE gender='Male' GROUP BY gender;",
    "sale amount earned from the purchase of male customers":"SELECT gender, SUM(total_amount) amount FROM retail_sales WHERE gender='Male' GROUP BY gender;",
    "total sale amount earned from the purchase of male customers":"SELECT gender, SUM(total_amount) amount FROM retail_sales WHERE gender='Male' GROUP BY gender;",
    "what is the sale amount earned from the purchase of male customers":"SELECT gender, SUM(total_amount) amount FROM retail_sales WHERE gender='Male' GROUP BY gender;",
    "what is the total sale amount earned from the purchase of male customers":"SELECT gender, SUM(total_amount) amount FROM retail_sales WHERE gender='Male' GROUP BY gender;",
    "what is the sale amount earned from the purchase of male customers?":"SELECT gender, SUM(total_amount) amount FROM retail_sales WHERE gender='Male' GROUP BY gender;",
    "what is the total sale amount earned from the purchase of male customers?":"SELECT gender, SUM(total_amount) amount FROM retail_sales WHERE gender='Male' GROUP BY gender;",
    
    "amount earned from the purchase of female customers":"SELECT gender, SUM(total_amount) amount FROM retail_sales WHERE gender='Female' GROUP BY gender;",
    "what is the amount earned from the purchase of female customers":"SELECT gender, SUM(total_amount) amount FROM retail_sales WHERE gender='Female' GROUP BY gender;",
    "what is the amount earned from the purchase of female customers?":"SELECT gender, SUM(total_amount) amount FROM retail_sales WHERE gender='Female' GROUP BY gender;",
    "amount earned from the purchase of female customers":"SELECT gender, SUM(total_amount) amount FROM retail_sales WHERE gender='Female' GROUP BY gender;",
    "what is the amount earned from the purchase of female customers":"SELECT gender, SUM(total_amount) amount FROM retail_sales WHERE gender='Female' GROUP BY gender;",
    "what is the amount earned from the purchase of female customers?":"SELECT gender, SUM(total_amount) amount FROM retail_sales WHERE gender='Female' GROUP BY gender;",
    "sale amount earned from the purchase of female customers":"SELECT gender, SUM(total_amount) amount FROM retail_sales WHERE gender='Female' GROUP BY gender;",
    "total sale amount earned from the purchase of female customers":"SELECT gender, SUM(total_amount) amount FROM retail_sales WHERE gender='Female' GROUP BY gender;",
    "what is the sale amount earned from the purchase of female customers":"SELECT gender, SUM(total_amount) amount FROM retail_sales WHERE gender='Female' GROUP BY gender;",
    "what is the total sale amount earned from the purchase of female customers":"SELECT gender, SUM(total_amount) amount FROM retail_sales WHERE gender='Female' GROUP BY gender;",
    "what is the sale amount earned from the purchase of female customers?":"SELECT gender, SUM(total_amount) amount FROM retail_sales WHERE gender='Female' GROUP BY gender;",
    "what is the total sale amount earned from the purchase of female customers?":"SELECT gender, SUM(total_amount) amount FROM retail_sales WHERE gender='Female' GROUP BY gender;",
    
    "amount earned from the purchase of male and female customers":"SELECT gender, SUM(total_amount) amount FROM retail_sales GROUP BY gender;",
    "what is the amount earned from the purchase of male and female customers":"SELECT gender, SUM(total_amount) amount FROM retail_sales GROUP BY gender;",
    "what is the amount earned from the purchase of male and female customers?":"SELECT gender, SUM(total_amount) amount FROM retail_sales GROUP BY gender;",
    "amount earned from the purchase of male and female customers":"SELECT gender, SUM(total_amount) amount FROM retail_sales GROUP BY gender;",
    "what is the amount earned from the purchase of male and female customers":"SELECT gender, SUM(total_amount) amount FROM retail_sales GROUP BY gender;",
    "what is the amount earned from the purchase of male and female customers?":"SELECT gender, SUM(total_amount) amount FROM retail_sales GROUP BY gender;",
    "sale amount earned from the purchase of male and female customers":"SELECT gender, SUM(total_amount) amount FROM retail_sales GROUP BY gender;",
    "total sale amount earned from the purchase of male and female customers":"SELECT gender, SUM(total_amount) amount FROM retail_sales GROUP BY gender;",
    "what is the sale amount earned from the purchase of male and female customers":"SELECT gender, SUM(total_amount) amount FROM retail_sales GROUP BY gender;",
    "what is the total sale amount earned from the purchase of male and female customers":"SELECT gender, SUM(total_amount) amount FROM retail_sales GROUP BY gender;",
    "what is the sale amount earned from the purchase of male and female customers?":"SELECT gender, SUM(total_amount) amount FROM retail_sales GROUP BY gender;",
    "what is the total sale amount earned from the purchase of male and female customers?":"SELECT gender, SUM(total_amount) amount FROM retail_sales GROUP BY gender;",
    "give me the amount earned from gender wise sale":"SELECT gender, SUM(total_amount) amount FROM retail_sales GROUP BY gender;",
    
    "what are the minimum and maximum amounts earned from one day sale":"SELECT MIN(total_amount) min_amount, MAX(total_amount) max_amount FROM retail_sales;",
    "what are the minimum and maximum amounts earned from one day sale?":"SELECT MIN(total_amount) min_amount, MAX(total_amount) max_amount FROM retail_sales;",
    "minimum and maximum amounts":"SELECT MIN(total_amount) min_amount, MAX(total_amount) max_amount FROM retail_sales;",
    "give me the minimum and maximum amounts earned":"SELECT MIN(total_amount) min_amount, MAX(total_amount) max_amount FROM retail_sales;",
    "minimum and maximum amounts earned":"SELECT MIN(total_amount) min_amount, MAX(total_amount) max_amount FROM retail_sales;",
    "give me the minimum and maximum amounts":"SELECT MIN(total_amount) min_amount, MAX(total_amount) max_amount FROM retail_sales;",
    
    "how many customers purchased clothings":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Clothing';",
    "how many customers purchased clothings?":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Clothing';",
    "no. of customers purchased clothings":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Clothing';",
    "number of customers purchased clothings":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Clothing';",
    "count of customers purchased clothings":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Clothing';",
    "what is the count of customers purchased clothings":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Clothing';",
    "count of sales in clothings category":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Clothing';",
    "what is the count of sales in clothings category":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Clothing';",
    
    "how many customers purchased electronics":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Electronics';",
    "how many customers purchased electronics?":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Electronics';",
    "no. of customers purchased electronics":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Electronics';",
    "number of customers purchased electronics":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Electronics';",
    "count of customers purchased electronics":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Electronics';",
    "what is the count of customers purchased electronics":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Electronics';",
    "count of sales in electronics category":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Electronics';",
    "what is the count of sales in electronics category":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Electronics';",
    
    "how many customers purchased beauty products":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Beauty'",
    "how many customers purchased beauty products?":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Beauty'",
    "no. of customers purchased beauty products":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Beauty'",
    "number of customers purchased beauty products":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Beauty'",
    "count of customers purchased beauty products":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Beauty'",
    "what is the count of customers purchased beauty products":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Beauty'",
    "count of sales in beauty category":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Beauty'",
    "what is the count of sales in beauty category":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Beauty'",
    
    "how many male customers purchased clothings":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Clothing' AND gender='Male';",
    "how many male customers purchased clothings?":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Clothing' AND gender='Male';",
    "no. of male customers purchased clothings":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Clothing' AND gender='Male';",
    "number of male customers purchased clothings":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Clothing' AND gender='Male';",
    "count of male customers purchased clothings":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Clothing' AND gender='Male';",
    "what is the count of male customers purchased clothings":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Clothing' AND gender='Male';",
    "count of sales in clothings category by male customers":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Clothing' AND gender='Male';",
    "what is the count of sales in clothings category by male customers":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Clothing' AND gender='Male';",
    
    "how many female customers purchased clothings":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Clothing' AND gender='Female';",
    "how many female customers purchased clothings?":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Clothing' AND gender='Female';",
    "no. of female customers purchased clothings":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Clothing' AND gender='Female';",
    "number of female customers purchased clothings":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Clothing' AND gender='Female';",
    "count of female customers purchased clothings":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Clothing' AND gender='Female';",
    "what is the count of female customers purchased clothings":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Clothing' AND gender='Female';",
    "count of sales in clothings category by female customers":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Clothing' AND gender='Female';",
    "what is the count of sales in clothings category by female customers":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Clothing' AND gender='Female';",
    
    "how many male customers purchased electronics":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Electronics' AND gender='Male';",
    "how many male customers purchased electronics?":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Electronics' AND gender='Male';",
    "no. of male customers purchased electronics":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Electronics' AND gender='Male';",
    "number of male customers purchased electronics":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Electronics' AND gender='Male';",
    "count of male customers purchased electronics":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Electronics' AND gender='Male';",
    "what is the count of male customers purchased electronics":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Electronics' AND gender='Male';",
    "count of sales in electronics category by male customers":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Electronics' AND gender='Male';",
    "what is the count of sales in electronics category by male customers":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Electronics' AND gender='Male';",
    
    "how many female customers purchased electronics":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Electronics' AND gender='Female';",
    "how many female customers purchased electronics?":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Electronics' AND gender='Female';",
    "no. of female customers purchased electronics":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Electronics' AND gender='Female';",
    "number of female customers purchased electronics":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Electronics' AND gender='Female';",
    "count of female customers purchased electronics":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Electronics' AND gender='Female';",
    "what is the count of female customers purchased electronics":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Electronics' AND gender='Female';",
    "count of sales in electronics category by female customers":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Electronics' AND gender='Female';",
    "what is the count of sales in electronics category by female customers":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE product_category='Electronics' AND gender='Female';",
    
    "how many male customers purchased beauty products":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Beauty' AND gender='Male';",
    "how many male customers purchased beauty products?":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Beauty' AND gender='Male';",
    "no. of male customers purchased beauty products":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Beauty' AND gender='Male';",
    "number of male customers purchased beauty products":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Beauty' AND gender='Male';",
    "count of male customers purchased beauty products":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Beauty' AND gender='Male';",
    "what is the count of male customers purchased beauty products":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Beauty' AND gender='Male';",
    "count of sales in beauty category by male customers":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Beauty' AND gender='Male';",
    "what is the count of sales in beauty category by male customers":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Beauty' AND gender='Male';",
    
    "how many female customers purchased beauty products":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Beauty' AND gender='Female';",
    "how many female customers purchased beauty products?":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Beauty' AND gender='Female';",
    "no. of female customers purchased beauty products":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Beauty' AND gender='Female';",
    "number of female customers purchased beauty products":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Beauty' AND gender='Female';",
    "count of female customers purchased beauty products":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Beauty' AND gender='Female';",
    "what is the count of female customers purchased beauty products":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Beauty' AND gender='Female';",
    "count of sales in beauty category by female customers":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Beauty' AND gender='Female';",
    "what is the count of sales in beauty category by female customers":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE product_category='Beauty' AND gender='Female';",
    
    "no. of customers purchased each product category":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales GROUP BY product_category ORDER BY sale_count DESC;",
    "number of customers purchased each product category":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales GROUP BY product_category ORDER BY sale_count DESC;",
    "no. of customers for each product category":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales GROUP BY product_category ORDER BY sale_count DESC;",
    "number of customers for each product category":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales GROUP BY product_category ORDER BY sale_count DESC;",
    "count of customers purchased each product category":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales GROUP BY product_category ORDER BY sale_count DESC;",
    "count of customers for each product category":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales GROUP BY product_category ORDER BY sale_count DESC;",
    "count of customers product category-wise":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales GROUP BY product_category ORDER BY sale_count DESC;",
    "no. of customers product category-wise":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales GROUP BY product_category ORDER BY sale_count DESC;",
    "number of customers product category-wise":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales GROUP BY product_category ORDER BY sale_count DESC;",
    "give me the no. of customers purchased in each product category":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales GROUP BY product_category ORDER BY sale_count DESC;",
    "give me the number of customers purchased in each product category":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales GROUP BY product_category ORDER BY sale_count DESC;",
    "give me the no. of customers for each product category":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales GROUP BY product_category ORDER BY sale_count DESC;",
    "give me the number of customers for each product category":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales GROUP BY product_category ORDER BY sale_count DESC;",
    "no. of customers in each category":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales GROUP BY product_category ORDER BY sale_count DESC;",
    "number of customers in each category":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales GROUP BY product_category ORDER BY sale_count DESC;",
    "count of customers in each category":"SELECT product_category, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales GROUP BY product_category ORDER BY sale_count DESC;",
    
    "no. of male customers purchased each product category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Male' GROUP BY product_category ORDER BY sale_count DESC;",
    "number of male customers purchased each product category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Male' GROUP BY product_category ORDER BY sale_count DESC;",
    "no. of male customers for each product category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Male' GROUP BY product_category ORDER BY sale_count DESC;",
    "number of male customers for each product category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Male' GROUP BY product_category ORDER BY sale_count DESC;",
    "count of male customers purchased each product category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Male' GROUP BY product_category ORDER BY sale_count DESC;",
    "count of male customers for each product category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Male' GROUP BY product_category ORDER BY sale_count DESC;",
    "count of male customers product category-wise":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Male' GROUP BY product_category ORDER BY sale_count DESC;",
    "no. of male customers product category-wise":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Male' GROUP BY product_category ORDER BY sale_count DESC;",
    "number of male customers product category-wise":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Male' GROUP BY product_category ORDER BY sale_count DESC;",
    "give me the no. of male customers purchased in each product category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Male' GROUP BY product_category ORDER BY sale_count DESC;",
    "give me the number of male customers purchased in each product category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Male' GROUP BY product_category ORDER BY sale_count DESC;",
    "give me the no. of male customers for each product category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Male' GROUP BY product_category ORDER BY sale_count DESC;",
    "give me the number of male customers for each product category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Male' GROUP BY product_category ORDER BY sale_count DESC;",
    "no. of male customers in each category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Male' GROUP BY product_category ORDER BY sale_count DESC;",
    "number of male customers in each category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Male' GROUP BY product_category ORDER BY sale_count DESC;",
    "count of male customers in each category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Male' GROUP BY product_category ORDER BY sale_count DESC;",
    
    "no. of female customers purchased each product category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Female' GROUP BY product_category ORDER BY sale_count DESC;",
    "number of female customers purchased each product category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Female' GROUP BY product_category ORDER BY sale_count DESC;",
    "no. of female customers for each product category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Female' GROUP BY product_category ORDER BY sale_count DESC;",
    "number of female customers for each product category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Female' GROUP BY product_category ORDER BY sale_count DESC;",
    "count of female customers purchased each product category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Female' GROUP BY product_category ORDER BY sale_count DESC;",
    "count of female customers for each product category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Female' GROUP BY product_category ORDER BY sale_count DESC;",
    "count of female customers product category-wise":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Female' GROUP BY product_category ORDER BY sale_count DESC;",
    "no. of female customers product category-wise":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Female' GROUP BY product_category ORDER BY sale_count DESC;",
    "number of female customers product category-wise":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Female' GROUP BY product_category ORDER BY sale_count DESC;",
    "give me the no. of female customers purchased in each product category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Female' GROUP BY product_category ORDER BY sale_count DESC;",
    "give me the number of female customers purchased in each product category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Female' GROUP BY product_category ORDER BY sale_count DESC;",
    "give me the no. of female customers for each product category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Female' GROUP BY product_category ORDER BY sale_count DESC;",
    "give me the number of female customers for each product category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Female' GROUP BY product_category ORDER BY sale_count DESC;",
    "no. of female customers in each category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Female' GROUP BY product_category ORDER BY sale_count DESC;",
    "number of female customers in each category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Female' GROUP BY product_category ORDER BY sale_count DESC;",
    "count of female customers in each category":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales WHERE gender='Female' GROUP BY product_category ORDER BY sale_count DESC;",
    
    "which product is purchased most by female customers":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE gender = 'Female' AND product_category = 'Clothing' GROUP BY product_category, gender ORDER BY sale_count DESC;",
    "female customers purchased the most":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE gender = 'Female' AND product_category = 'Clothing' GROUP BY product_category, gender ORDER BY sale_count DESC;",
    "product which the female customers prefer the most":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE gender = 'Female' AND product_category = 'Clothing' GROUP BY product_category, gender ORDER BY sale_count DESC;",
    "product category that is highly purchased by female customers":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE gender = 'Female' AND product_category = 'Clothing' GROUP BY product_category, gender ORDER BY sale_count DESC;",
    "product category that female customers purchased the most":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE gender = 'Female' AND product_category = 'Clothing' GROUP BY product_category, gender ORDER BY sale_count DESC;",
    "product category that females purchased the most":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE gender = 'Female' AND product_category = 'Clothing' GROUP BY product_category, gender ORDER BY sale_count DESC;",
    
    "which product is purchased most by male customers":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE gender = 'Male' AND product_category = 'Clothing' GROUP BY product_category, gender ORDER BY sale_count DESC;",
    "male customers purchased the most":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE gender = 'Male' AND product_category = 'Clothing' GROUP BY product_category, gender ORDER BY sale_count DESC;",
    "product which the male customers prefer the most":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE gender = 'Male' AND product_category = 'Clothing' GROUP BY product_category, gender ORDER BY sale_count DESC;",
    "product category that is highly purchased by male customers":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE gender = 'Male' AND product_category = 'Clothing' GROUP BY product_category, gender ORDER BY sale_count DESC;",
    "product category that male customers purchased the most":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE gender = 'Male' AND product_category = 'Clothing' GROUP BY product_category, gender ORDER BY sale_count DESC;",
    "product category that males purchased the most":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  WHERE gender = 'Male' AND product_category = 'Clothing' GROUP BY product_category, gender ORDER BY sale_count DESC;",
    
    "month with highest sale":"SELECT MONTH(date) month, YEAR(date) year, COUNT(DISTINCT transaction_id) sale_count, SUM(total_amount) sale_amount FROM retail_sales GROUP BY 1,2 ORDER BY 4 DESC;",
    "month with highest revenue":"SELECT MONTH(date) month, YEAR(date) year, COUNT(DISTINCT transaction_id) sale_count, SUM(total_amount) sale_amount FROM retail_sales GROUP BY 1,2 ORDER BY 4 DESC;",
    "which month has highest sale":"SELECT MONTH(date) month, YEAR(date) year, COUNT(DISTINCT transaction_id) sale_count, SUM(total_amount) sale_amount FROM retail_sales GROUP BY 1,2 ORDER BY 4 DESC;",
    "which month has highest revenue":"SELECT MONTH(date) month, YEAR(date) year, COUNT(DISTINCT transaction_id) sale_count, SUM(total_amount) sale_amount FROM retail_sales GROUP BY 1,2 ORDER BY 4 DESC;",
    "month-wise sale":"SELECT MONTH(date) month, YEAR(date) year, COUNT(DISTINCT transaction_id) sale_count, SUM(total_amount) sale_amount FROM retail_sales GROUP BY 1,2 ORDER BY 4 DESC;",
    "month-wise revenue":"SELECT MONTH(date) month, YEAR(date) year, COUNT(DISTINCT transaction_id) sale_count, SUM(total_amount) sale_amount FROM retail_sales GROUP BY 1,2 ORDER BY 4 DESC;",
    
    "no. of customers categorised by products and gender":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  GROUP BY product_category, gender ORDER BY sale_count DESC;",
    "number of customers categorised by products and gender":"SELECT product_category, gender, COUNT(DISTINCT transaction_id) sale_count FROM retail_sales  GROUP BY product_category, gender ORDER BY sale_count DESC;"
    
}

In [3]:
class ChatBotGUI:
    def __init__(self, master):
        self.master = master
        master.title("SQLChat Buddy")

        # Customize the appearance of the GUI
        master.geometry("800x600")
        master.configure(bg="#f0f0f0")

        # Create text area for displaying conversation
        self.text_area = scrolledtext.ScrolledText(master, wrap=tk.WORD, width=80, height=20, bg="#ffffff", fg="#000000", font=("Arial", 12))
        self.text_area.pack(padx=10, pady=10)

        # Create entry widget for user input
        self.entry = tk.Entry(master, width=80, bg="#ffffff", fg="#000000", font=("Arial", 12))
        self.entry.pack(pady=5)

        # Create send button
        self.button = tk.Button(master, text="Send", command=self.send_message, bg="#4caf50", fg="#ffffff", font=("Arial", 12))
        self.button.pack(pady=5)

        # Create a label for additional information
        self.info_label = tk.Label(master, text="Type your query and press 'Send' to get the response.", bg="#f0f0f0", fg="#000000", font=("Arial", 12))
        self.info_label.pack(pady=5)

        # Establish database connection
        self.db_connection = mysql.connector.connect(
            host="127.0.0.1",
            user="root",
            passwd="1702",
            database="retail"
        )

    def send_message(self):
        user_input = self.entry.get()
        response = self.process_query(user_input)
        self.display_message(f"You: {user_input}\n", bold=False)
        self.display_message(f"ChatBot: {response}\n\n", bold=False)
        self.entry.delete(0, tk.END)

    def process_query(self, query):
        try:
            cursor = self.db_connection.cursor()

            # Check if the query directly matches any key in the conversation dictionary
            if query in conversation:
                sql_query = conversation[query]
            else:
                # Use get_close_matches to find the closest match in the conversation dictionary
                closest_match = get_close_matches(query, conversation.keys(), n=1)
                if closest_match:
                    sql_query = conversation[closest_match[0]]
                else:
                    return "Sorry, I don't understand."

            cursor.execute(sql_query)
            result = cursor.fetchall()
            response = self.format_result(result, cursor)

            cursor.close()
            return response
        except mysql.connector.Error as e:
            if e.errno == 1064:
                return "Sorry, I don't understand."
            else:
                return f"Error: {e}"
        except Exception as e:
            return f"Error: {e}"

    def format_result(self, result, cursor):
        if not result:
            return "No results found."

        # Get column names
        column_names = [desc[0] for desc in cursor.description]

        # Determine column widths
        column_widths = [max(len(str(row[i])) for row in result) for i in range(len(column_names))]

        # Create header row
        header = " | ".join(f"{column_names[i]:<{column_widths[i]}}" for i in range(len(column_names)))

        # Create rows of data
        rows = [header]
        for row in result:
            rows.append(" | ".join(f"{str(row[i]):<{column_widths[i]}}" for i in range(len(column_names))))

        return "\n".join(rows)

    def display_message(self, message, bold=False):
        self.text_area.configure(state=tk.NORMAL)
        self.text_area.insert(tk.END, message, "bold" if bold else "")
        self.text_area.configure(state=tk.DISABLED)

def main():
    root = tk.Tk()
    gui = ChatBotGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()
